In [1]:
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Load data

In [5]:
adult_oh = pd.read_csv('adult_oh.csv')
adult_oh.head()

,age,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,...,native_country_ Puerto-Rico,native_country_ Scotland,native_country_ South,native_country_ Taiwan,native_country_ Thailand,native_country_ Trinadad&Tobago,native_country_ United-States,native_country_ Vietnam,native_country_ Yugoslavia,Y
0,39,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,50,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,38,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,53,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,28,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Create 5 trials, each with train and test set


In [6]:
from sklearn.model_selection import train_test_split

XY_train1, XY_test1 = train_test_split(adult_oh, train_size=5000, shuffle=True)
XY_train2, XY_test2 = train_test_split(adult_oh, train_size=5000, shuffle=True)
XY_train3, XY_test3 = train_test_split(adult_oh, train_size=5000, shuffle=True)
XY_train4, XY_test4 = train_test_split(adult_oh, train_size=5000, shuffle=True)
XY_train5, XY_test5 = train_test_split(adult_oh, train_size=5000, shuffle=True)

In [7]:
print(XY_train1.shape)
print(XY_test1.shape)
XY_train1.head()

(5000, 109)
(27561, 109)


,age,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,...,native_country_ Puerto-Rico,native_country_ Scotland,native_country_ South,native_country_ Taiwan,native_country_ Thailand,native_country_ Trinadad&Tobago,native_country_ United-States,native_country_ Vietnam,native_country_ Yugoslavia,Y
5493,25,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
328,58,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
8368,31,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
14717,63,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
16777,45,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


## Perform GridSearchCV on classifier


In [8]:
%%time
#import warnings
# there are a lot of convergence warnings for some params, however be careful with this!!
# sometimes you need to see those wanrings, and now we've screwed tha tup for the whole notebook from here on!!
#warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV

# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', KNeighborsClassifier())])

# Create search space of candidate learning algorithms and their hyperparameters
grid_params = {
    'n_neighbors': [3,5,11,19],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

trialnum = 0
accuracy_sum = 0 # sum of top accuracy to later calculate the average of all 5 trials
roc_sum = 0 # sum of top roc score to later calculate the average of all 5 trials
f1_sum = 0 # sum of top accuracy to later calculate the average of all 5 trials
accuracy_scores = []
roc_scores = []
f1_scores = []
all_accuracy_models = []
all_roc_models = []
all_f1_models = []

# for every trial
for trial in [XY_train1, XY_train2, XY_train3, XY_train4, XY_train5]:
 
    trialnum = trialnum + 1
    X_l = trial.drop(['Y'],1)
    y_l = trial['Y']

    clf = GridSearchCV(KNeighborsClassifier(), param_grid=grid_params, cv=StratifiedKFold(n_splits=5), 
                      scoring=['accuracy', 'roc_auc_ovr', 'f1_micro'], refit=False, 
                       n_jobs=-1, verbose=-1)

    # Fit grid search
    best_model = clf.fit(X_l, y_l)
    
    print("------------------------------------------------------------------------------------")
    print("RESULTS FOR TRIAL:")
    print(trialnum)
    print("------------------------------------------------------------------------------------")
    
    # the detailed results of the whole model selection search...
#     print(best_model.cv_results_)

    print("---------------BEST MODEL FOR ACCURACY: ----------")
    print( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_accuracy']) ] )
    print("---WITH ACCURACY: ---")
    current_accuracy = best_model.cv_results_['mean_test_accuracy'][ np.argmax(best_model.cv_results_['mean_test_accuracy']) ]
    print(current_accuracy)
    accuracy_sum = accuracy_sum + current_accuracy
    accuracy_scores.append(current_accuracy)
    all_accuracy_models.append( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_accuracy']) ] )
    
    print("---------------BEST MODEL FOR ROC: ---------------")
    print( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_roc_auc_ovr']) ] )
    print("---WITH ROC: ---")
    current_roc = best_model.cv_results_['mean_test_roc_auc_ovr'][ np.argmax(best_model.cv_results_['mean_test_roc_auc_ovr']) ]
    print(current_roc)
    roc_sum = roc_sum + current_roc
    roc_scores.append(current_roc)
    all_roc_models.append( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_roc_auc_ovr']) ] )

    
    print("---------------BEST MODEL FOR F1: ----------------")
    print( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro']) ] )
    print("---WITH F1: ---")    
    current_f1 = best_model.cv_results_['mean_test_f1_micro'][ np.argmax(best_model.cv_results_['mean_test_f1_micro']) ]
    print(current_f1)
    f1_sum = f1_sum + current_f1
    f1_scores.append(current_f1)
    all_f1_models.append( best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro']) ] )


------------------------------------------------------------------------------------
RESULTS FOR TRIAL:
1
------------------------------------------------------------------------------------
---------------BEST MODEL FOR ACCURACY: ----------
{'metric': 'euclidean', 'n_neighbors': 19, 'weights': 'uniform'}
---WITH ACCURACY: ---
0.7814
---------------BEST MODEL FOR ROC: ---------------
{'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'distance'}
---WITH ROC: ---
0.6139841088199857
---------------BEST MODEL FOR F1: ----------------
{'metric': 'euclidean', 'n_neighbors': 19, 'weights': 'uniform'}
---WITH F1: ---
0.7814000000000001
------------------------------------------------------------------------------------
RESULTS FOR TRIAL:
2
------------------------------------------------------------------------------------
---------------BEST MODEL FOR ACCURACY: ----------
{'metric': 'euclidean', 'n_neighbors': 11, 'weights': 'uniform'}
---WITH ACCURACY: ---
0.775
---------------BEST MODEL

### Get Train Metrics

In [9]:
print("===== ACCURACY SCORES: =====")
print(accuracy_scores)
print("===== ROC SCORES: =====")
print(roc_scores)
print("===== F1 SCORES: =====")
print(f1_scores)

===== ACCURACY SCORES: =====
[0.7814, 0.775, 0.7878000000000001, 0.7790000000000001, 0.779]
===== ROC SCORES: =====
[0.6139841088199857, 0.6190501606607579, 0.627976252807197, 0.643225954004419, 0.620734126984127]
===== F1 SCORES: =====
[0.7814000000000001, 0.775, 0.7878000000000001, 0.7790000000000001, 0.7790000000000001]


In [10]:
print("================ BEST ACCURACY MODEL IN TRAINING: ==================")
print(all_accuracy_models[ np.argmax(accuracy_scores) ])
print("WITH ACCURACY:")
print(max(accuracy_scores))
print("================ BEST ROC MODEL IN TRAINING: ==================")
print(all_roc_models[ np.argmax(roc_scores) ])
print("WITH ROC SCORE:")
print(max(roc_scores))
print("================ BEST F1 MODEL IN TRAINING: ==================")
print(all_f1_models[ np.argmax(f1_scores) ])
print("WITH F1 SCORE:")
print(max(f1_scores))

================ BEST ACCURACY MODEL IN TRAINING: ==================
{'metric': 'euclidean', 'n_neighbors': 11, 'weights': 'uniform'}
WITH ACCURACY:
0.7878000000000001
================ BEST ROC MODEL IN TRAINING: ==================
{'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
WITH ROC SCORE:
0.643225954004419
================ BEST F1 MODEL IN TRAINING: ==================
{'metric': 'euclidean', 'n_neighbors': 11, 'weights': 'uniform'}
WITH F1 SCORE:
0.7878000000000001


In [11]:
print("================ AVERAGE ACCURACY ON TRAIN SET: ==================")
print(accuracy_sum / 5)
print("================ AVERAGE ROC SCORE ON TRAIN SET: ==================")
print(roc_sum / 5)
print("================ AVERAGE F1 SCORE ON TRAIN SET: ==================")
print(f1_sum / 5)

================ AVERAGE ACCURACY ON TRAIN SET: ==================
0.7804399999999999
================ AVERAGE ROC SCORE ON TRAIN SET: ==================
0.6249941206552972
================ AVERAGE F1 SCORE ON TRAIN SET: ==================
0.7804399999999999


### Get Test Metrics

In [12]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

# test performance of model rated as best for accuracy and f1 score on training set
performance_AccModel = pd.DataFrame(index=['acc', 'roc', 'f1'], columns=['Trial 1', 'Trial 2', 'Trial 3', 'Trial 4', 'Trial 5'])
performance_AccModel

# test performance on trial 1 test set
clf = KNeighborsClassifier(metric='euclidean', n_neighbors=11, weights='uniform').fit(XY_train1.drop(['Y'],1), XY_train1['Y'])
pred = clf.predict(XY_test1.drop(['Y'],1))
performance_AccModel.loc['acc', 'Trial 1'] = accuracy_score(XY_test1['Y'], pred)
performance_AccModel.loc['roc', 'Trial 1'] = roc_auc_score(XY_test1['Y'], pred)
performance_AccModel.loc['f1', 'Trial 1'] = f1_score(XY_test1['Y'], pred)

# test performance on trial 2 test set
clf = KNeighborsClassifier(metric='euclidean', n_neighbors=11, weights='uniform').fit(XY_train2.drop(['Y'],1), XY_train2['Y'])
pred = clf.predict(XY_test2.drop(['Y'],1))
performance_AccModel.loc['acc', 'Trial 2'] = accuracy_score(XY_test2['Y'], pred)
performance_AccModel.loc['roc', 'Trial 2'] = roc_auc_score(XY_test2['Y'], pred)
performance_AccModel.loc['f1', 'Trial 2'] = f1_score(XY_test2['Y'], pred)

# test performance on trial 3 test set
clf = KNeighborsClassifier(metric='euclidean', n_neighbors=11, weights='uniform').fit(XY_train3.drop(['Y'],1), XY_train3['Y'])
pred = clf.predict(XY_test3.drop(['Y'],1))
performance_AccModel.loc['acc', 'Trial 3'] = accuracy_score(XY_test3['Y'], pred)
performance_AccModel.loc['roc', 'Trial 3'] = roc_auc_score(XY_test3['Y'], pred)
performance_AccModel.loc['f1', 'Trial 3'] = f1_score(XY_test3['Y'], pred)

# test performance on trial 4 test set
clf = KNeighborsClassifier(metric='euclidean', n_neighbors=11, weights='uniform').fit(XY_train4.drop(['Y'],1), XY_train4['Y'])
pred = clf.predict(XY_test4.drop(['Y'],1))
performance_AccModel.loc['acc', 'Trial 4'] = accuracy_score(XY_test4['Y'], pred)
performance_AccModel.loc['roc', 'Trial 4'] = roc_auc_score(XY_test4['Y'], pred)
performance_AccModel.loc['f1', 'Trial 4'] = f1_score(XY_test4['Y'], pred)

# test performance on trial 5 test set
clf = KNeighborsClassifier(metric='euclidean', n_neighbors=11, weights='uniform').fit(XY_train5.drop(['Y'],1), XY_train5['Y'])
pred = clf.predict(XY_test5.drop(['Y'],1))
performance_AccModel.loc['acc', 'Trial 5'] = accuracy_score(XY_test5['Y'], pred)
performance_AccModel.loc['roc', 'Trial 5'] = roc_auc_score(XY_test5['Y'], pred)
performance_AccModel.loc['f1', 'Trial 5'] = f1_score(XY_test5['Y'], pred)

performance_AccModel

,Trial 1,Trial 2,Trial 3,Trial 4,Trial 5
acc,0.786836,0.786111,0.785966,0.789449,0.784514
roc,0.573996,0.577239,0.573679,0.581234,0.567976
f1,0.269369,0.282759,0.269474,0.292403,0.252109


In [13]:
# test performance of model rated as best for roc on training set
performance_ROCModel = pd.DataFrame(index=['acc', 'roc', 'f1'], columns=['Trial 1', 'Trial 2', 'Trial 3', 'Trial 4', 'Trial 5'])

# test performance on trial 1 test set
clf = KNeighborsClassifier(metric='manhattan', n_neighbors=3, weights='distance').fit(XY_train1.drop(['Y'],1), XY_train1['Y'])
pred = clf.predict(XY_test1.drop(['Y'],1))
performance_ROCModel.loc['acc', 'Trial 1'] = accuracy_score(XY_test1['Y'], pred)
performance_ROCModel.loc['roc', 'Trial 1'] = roc_auc_score(XY_test1['Y'], pred)
performance_ROCModel.loc['f1', 'Trial 1'] = f1_score(XY_test1['Y'], pred)

# test performance on trial 2 test set
clf = KNeighborsClassifier(metric='manhattan', n_neighbors=3, weights='distance').fit(XY_train2.drop(['Y'],1), XY_train2['Y'])
pred = clf.predict(XY_test2.drop(['Y'],1))
performance_ROCModel.loc['acc', 'Trial 2'] = accuracy_score(XY_test2['Y'], pred)
performance_ROCModel.loc['roc', 'Trial 2'] = roc_auc_score(XY_test2['Y'], pred)
performance_ROCModel.loc['f1', 'Trial 2'] = f1_score(XY_test2['Y'], pred)

# test performance on trial 3 test set
clf = KNeighborsClassifier(metric='manhattan', n_neighbors=3, weights='distance').fit(XY_train3.drop(['Y'],1), XY_train3['Y'])
pred = clf.predict(XY_test3.drop(['Y'],1))
performance_ROCModel.loc['acc', 'Trial 3'] = accuracy_score(XY_test3['Y'], pred)
performance_ROCModel.loc['roc', 'Trial 3'] = roc_auc_score(XY_test3['Y'], pred)
performance_ROCModel.loc['f1', 'Trial 3'] = f1_score(XY_test3['Y'], pred)

# test performance on trial 4 test set
clf = KNeighborsClassifier(metric='manhattan', n_neighbors=3, weights='distance').fit(XY_train4.drop(['Y'],1), XY_train4['Y'])
pred = clf.predict(XY_test4.drop(['Y'],1))
performance_ROCModel.loc['acc', 'Trial 4'] = accuracy_score(XY_test4['Y'], pred)
performance_ROCModel.loc['roc', 'Trial 4'] = roc_auc_score(XY_test4['Y'], pred)
performance_ROCModel.loc['f1', 'Trial 4'] = f1_score(XY_test4['Y'], pred)

# test performance on trial 5 test set
clf = KNeighborsClassifier(metric='manhattan', n_neighbors=3, weights='distance').fit(XY_train5.drop(['Y'],1), XY_train5['Y'])
pred = clf.predict(XY_test5.drop(['Y'],1))
performance_ROCModel.loc['acc', 'Trial 5'] = accuracy_score(XY_test5['Y'], pred)
performance_ROCModel.loc['roc', 'Trial 5'] = roc_auc_score(XY_test5['Y'], pred)
performance_ROCModel.loc['f1', 'Trial 5'] = f1_score(XY_test5['Y'], pred)

performance_ROCModel

,Trial 1,Trial 2,Trial 3,Trial 4,Trial 5
acc,0.735532,0.732448,0.735206,0.735532,0.735024
roc,0.60038,0.608045,0.603349,0.606895,0.605068
f1,0.382236,0.398139,0.388367,0.394652,0.391569


## Get final KNN results

In [18]:
print("================ BEST ACCURACY MODEL IN KNN: ==================")
print(all_accuracy_models[ np.argmax(np.array(performance_AccModel.iloc[0])) ])
print("WITH ACCURACY:")
print((np.array(performance_AccModel.iloc[0])).mean())
print("================ BEST ROC MODEL IN KNN: ==================")
print(all_roc_models[ np.argmax(np.array(performance_ROCModel.iloc[1])) ])
print("WITH ROC SCORE:")
print((performance_ROCModel.iloc[1]).mean())
print("================ BEST F1 MODEL IN KNN: ==================")
print(all_roc_models[ np.argmax(np.array(performance_ROCModel.iloc[2])) ])
print("WITH F1 SCORE:")
print((np.array(performance_ROCModel.iloc[2])).mean())

================ BEST ACCURACY MODEL IN KNN: ==================
{'metric': 'euclidean', 'n_neighbors': 11, 'weights': 'uniform'}
WITH ACCURACY:
0.7865752331192628
================ BEST ROC MODEL IN KNN: ==================
{'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}
WITH ROC SCORE:
0.6047475875923892
================ BEST F1 MODEL IN KNN: ==================
{'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}
WITH F1 SCORE:
0.39099253177270776
